In [4]:
pip install mediapipe


  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/c1/0f/4dc0802131756a9fe4d46d2824352014b85a75baca386cb9e43057f39f15/mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata
  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/7e/cf/5f51b43bd692e90585c0ef6e8d1b0db5d254fe0224a6570daa59a1be014f/jax-0.4.31-py3-none-any.whl.metadata
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/9f/e2/5b7d20ed550d156311587eee6e44c48971fe6c3b43f39e82dacda3875396/jaxlib-0.4.31-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/a7/9e/7110d2c5d543ab03b9581dbb1f8e2429863e44e0c9b4960b766f230c1279/opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\mrina\\Anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import os
import cv2
import shutil
import time
import json


def vidToFrame(vid_file,out_dir):


    video_capture = cv2.VideoCapture(vid_file)

    output_folder = out_dir
    #shutil.rmtree(output_folder)
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Initialize frame count and flag
    vid_cnt = 0
    success = True



    # Read the next frame
    #success, frame = video_capture.read()

    
    frame_count=0
    while frame_count < 48:
        
        frame_count += 1
        success, frame = video_capture.read()
        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")

        if frame is not None:
            cv2.imwrite(frame_path, frame)

        
#        cv2.imshow('frame', frame)
        
        if cv2.waitKey(20) & 0xFF == ord('q') :
            break

    # Release the video capture object
    video_capture.release()
    cv2.destroyAllWindows()





file_path = 'D:/endtoend/gesturerecognition1/wlasl/WLASL_v0.3.json'
missing_file_path = 'D:/endtoend/gesturerecognition1/wlasl/missing.txt'
videos_dir = 'D:/endtoend/gesturerecognition1/wlasl/videos'

# Load the WLASL dataset
with open(file_path) as file:
    wlasl = json.load(file)

# Read the missing video IDs from the file
with open(missing_file_path, 'r') as file:
    missing_videos = file.read().splitlines()

# Specify the base directory for the dataset
dataset_dir = '/kaggle/working/Dataset/frames'

# Create necessary directories
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'Train'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'Test'), exist_ok=True)

# Process each class in the WLASL dataset
for class_data in wlasl:
    class_name = class_data['gloss']
    print(class_name)
    
    for instance in class_data['instances']:
        video_id = instance['video_id']
        
        if video_id not in missing_videos:
            video_file = os.path.join(videos_dir, video_id + '.mp4')
    
            if instance['split'] == 'train':
                train_dir = os.path.join(dataset_dir, 'Train', class_name, video_id)
                os.makedirs(train_dir, exist_ok=True)
                vidToFrame(video_file, train_dir)
                print('train', video_id)
            else:
                test_dir = os.path.join(dataset_dir, 'Test', class_name, video_id)
                os.makedirs(test_dir, exist_ok=True)
                vidToFrame(video_file, test_dir)
                print('test', video_id)

book
train 69241
train 07069
train 07068
train 07070
test 07099
train 07074
drink
test 69302
train 65539
train 17710
train 17733
train 65540
test 17734
train 17711
train 17712
test 17713
train 17709
train 17720
train 17721
train 17722
train 17723
test 17724
computer
train 12328
test 12312
train 12311
train 12338
train 12313
train 12314
test 12315
train 12316
train 12317
train 12318
train 12319
test 12320
test 12326
train 12327
before
train 05728
train 05749
test 05750
train 05729
train 05730
train 65167
train 05731
train 05732
train 05733
train 05734
test 05727
test 05739
train 05740
test 05741
train 05742
train 05743
chair
train 09848
train 09869
train 09849
train 09850
test 09851
train 65328
test 09854
go
test 69345
train 24955
train 24956
train 24941
test 24960
train 24961
train 24962
train 65824
train 24973
train 24943
test 24946
train 24947
train 24940
test 24952
train 24954
clothes
train 11310
train 11330
test 11311
train 11313
train 11309
who
train 69534
train 63242
test 63226
t

In [18]:
import mediapipe as mp
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import os
import shutil
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
import cv2
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
np.random.seed(42)



# Create a Holistic object to detect pose, face, and hands keypoints
mp_holistic = mp.solutions.holistic

# Drawing utilities
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    if image is None:
        raise ValueError(f"Failed to load image: {image}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

def adjust_landmarks(arr,center):

    # Reshape the array to have shape (n, 3)
    arr_reshaped = arr.reshape(-1, 3)

    # Repeat the center array to have shape (n, 3)
    center_repeated = np.tile(center, (len(arr_reshaped), 1))

    # Subtract the center array from the arr array
    arr_adjusted = arr_reshaped - center_repeated

    # Reshape arr_adjusted back to shape (n*3,)
    arr_adjusted = arr_adjusted.reshape(-1)
    return(arr_adjusted)

def extract_keypoints(results):

    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    nose=pose[:3]
    lh_wrist=lh[:3]
    rh_wrist=rh[:3]
    pose_adjusted = adjust_landmarks(pose,nose)
    lh_adjusted = adjust_landmarks(lh,lh_wrist)
    rh_adjusted = adjust_landmarks(rh,rh_wrist)
    return pose_adjusted, lh_adjusted, rh_adjusted

selected_words = os.listdir('/kaggle/working/Dataset/frames/Train')

def make_keypoint_arrays(path,split):

    os.makedirs('/kaggle/working/Dataset/npy_arrays',exist_ok = True)
    os.makedirs(f'/kaggle/working/Dataset/npy_arrays/{split}',exist_ok = True)
    working_path = f'/kaggle/working/Dataset/npy_arrays/{split}'
    words_folder = os.path.join(path, split)
    selected_words1=[]
    for words1 in selected_words:
        npy_fold = os.listdir(os.path.join(working_path))
        if words1 not in npy_fold:
            selected_words1.append(words1)
    i=1
    # Loop through all the subfolders in the folder
    for word in tqdm(selected_words1):
        npy_fold = os.listdir(os.path.join(working_path))
        if word not in npy_fold:
            video_files = os.listdir(os.path.join(words_folder, word))
              # Loop through the video files
            for video_file in video_files:
                    # Open the video file
                video = sorted(os.listdir(os.path.join(words_folder, word, video_file)))
        
                # Initialize the list of keypoints for this video
                pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                  # Loop through the video frames
                  for frame in video:
                      # Perform any necessary preprocessing on the frame (e.g., resizing, normalization)
                    frame = os.path.join(words_folder, word, video_file,frame)
                    frame = cv2.imread(frame)
        #                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    
                      # Normalize pixel values to the range [0, 1]
                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
        
                    # Extract keypoints
                    pose, lh, rh = extract_keypoints(results)
                    # Add the keypoints to the list for this video
                    pose_keypoints.append(pose)
                    lh_keypoints.append(lh)
                    rh_keypoints.append(rh)           
                    # Save the keypoints for this video to a numpy array
                    pose_directory = os.path.join(working_path, word,'pose_keypoints')
                    lh_directory = os.path.join(working_path, word,'lh_keypoints')
                    rh_directory = os.path.join(working_path, word,'rh_keypoints')
        
                    if not os.path.exists(pose_directory):
                        os.makedirs(pose_directory)
        
                    if not os.path.exists(lh_directory):
                        os.makedirs(lh_directory)
        
                    if not os.path.exists(rh_directory):
                        os.makedirs(rh_directory)
        
                    pose_path = os.path.join(pose_directory, video_file)
                    np.save(pose_path, pose_keypoints)
        
                    lh_path = os.path.join(lh_directory, video_file)
                    np.save(lh_path, lh_keypoints)
        
                    rh_path = os.path.join(rh_directory, video_file)
                    np.save(rh_path, rh_keypoints)

make_keypoint_arrays('/kaggle/working/Dataset/frames','Train/')


words= np.array(os.listdir('/kaggle/working/Dataset/frames/Train'))
print(words)

label_map = {label:num for num, label in enumerate(words)}
print(label_map)

def preprocess_data(data_path,split,f_avg):

    # Initialize the lists of sequences and labels
    sequences = []
    labels = []
    
    # Iterate over the words
    for word in tqdm(words):
        word_path = os.path.join(data_path, split, word)
        word_id = os.path.basename(word_path).zfill(4)
    
        lh_keypoints_folder = os.path.join(word_path, "lh_keypoints")
        rh_keypoints_folder = os.path.join(word_path, "rh_keypoints")
        pose_keypoints_folder = os.path.join(word_path, "pose_keypoints")
    
        # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
        for sequence in os.listdir(lh_keypoints_folder):
            # Load the left hand array
            res_lh = np.load(os.path.join(lh_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_lh.shape[0], f_avg)
            res_lh = res_lh[:num_frames, :]
            while num_frames < f_avg:
                res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
                num_frames += 1
    
            # Load the right hand array
            res_rh = np.load(os.path.join(rh_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_rh.shape[0], f_avg)
            res_rh = res_rh[:num_frames, :]
            while num_frames < f_avg:
                res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
                num_frames += 1
    
            # Load the pose array
            res_pose = np.load(os.path.join(pose_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_pose.shape[0], f_avg)
            res_pose = res_pose[:num_frames, :]
            while num_frames < f_avg:
                res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
                num_frames += 1
    
            # Append the subsequence to the list of sequences
            sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
            # Append the label to the list of labels
            labels.append(label_map[word])
    
    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    y = to_categorical(labels).astype(int)
    
    return X, y

#train and validation splits
data_path = '/kaggle/working/Dataset/npy_arrays'

X_train,y_train=preprocess_data(data_path,'Train/',48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)


# Define the Bidirectional LSTM model with Attention
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(words), activation='softmax')
])

# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model_training_history = model.fit(X_train, y_train, epochs=50)





0it [00:00, ?it/s]


['a' 'a lot' 'abdomen' ... 'your' 'yourself' 'zero']
{'a': 0, 'a lot': 1, 'abdomen': 2, 'able': 3, 'about': 4, 'above': 5, 'accent': 6, 'accept': 7, 'accident': 8, 'accomplish': 9, 'accountant': 10, 'across': 11, 'act': 12, 'action': 13, 'active': 14, 'activity': 15, 'actor': 16, 'adapt': 17, 'add': 18, 'address': 19, 'adjective': 20, 'adjust': 21, 'admire': 22, 'admit': 23, 'adopt': 24, 'adult': 25, 'advanced': 26, 'advantage': 27, 'adverb': 28, 'affect': 29, 'afraid': 30, 'africa': 31, 'after': 32, 'afternoon': 33, 'again': 34, 'against': 35, 'age': 36, 'agenda': 37, 'ago': 38, 'agree': 39, 'agreement': 40, 'ahead': 41, 'aid': 42, 'aim': 43, 'airplane': 44, 'alarm': 45, 'alcohol': 46, 'algebra': 47, 'all': 48, 'all day': 49, 'allergy': 50, 'alligator': 51, 'allow': 52, 'almost': 53, 'alone': 54, 'alphabet': 55, 'already': 56, 'also': 57, 'always': 58, 'amazing': 59, 'america': 60, 'amputate': 61, 'analyze': 62, 'anatomy': 63, 'and': 64, 'angel': 65, 'angle': 66, 'angry': 67, 'animal'

100%|██████████| 1998/1998 [00:13<00:00, 143.36it/s]


(6650, 48, 225)
(6650, 1998)
(1663, 48, 225)
(1663, 1998)
Epoch 1/50
208/208 [==============================] - 16s 48ms/step - loss: 7.6053 - accuracy: 7.5188e-04
Epoch 2/50
208/208 [==============================] - 10s 46ms/step - loss: 7.5900 - accuracy: 9.0226e-04
Epoch 3/50
208/208 [==============================] - 10s 48ms/step - loss: 7.5845 - accuracy: 0.0014
Epoch 4/50
208/208 [==============================] - 10s 49ms/step - loss: 7.5795 - accuracy: 0.0014
Epoch 5/50
208/208 [==============================] - 10s 47ms/step - loss: 7.5749 - accuracy: 0.0017
Epoch 6/50
208/208 [==============================] - 10s 49ms/step - loss: 7.5705 - accuracy: 0.0017
Epoch 7/50
208/208 [==============================] - 10s 49ms/step - loss: 7.5665 - accuracy: 0.0017
Epoch 8/50
208/208 [==============================] - 10s 48ms/step - loss: 7.5628 - accuracy: 0.0017
Epoch 9/50
208/208 [==============================] - 10s 48ms/step - loss: 7.5593 - accuracy: 0.0014
Epoch 10/50
208/